# HSE-2024 text classification project

## Table of content
* [Imports](#c1)
* [Data loading and preprocessing](#c2)
* [Custom dataset](#c3)
* [Custom model](#c4)
* [Trainer](#c5)
* [BERT training](#c6)
* [GPT training](#c6)

### Imports <a class="anchor" id="c1"></a>

In [1]:
import numpy as np
import pandas as pd

import torch
from sklearn.model_selection import train_test_split

import os
import re
import string

from dicts import var_vocab, args, topic_to_russian
from classes import Preprocessor, MyModel, Trainer

### Data loading and preprocessing <a class="anchor" id="c2"></a>

In [2]:
RANDOM_STATE = 42
torch.manual_seed(RANDOM_STATE)

In [3]:
base_dir = 'data/'
if not os.path.exists(base_dir):  # создадим папку, куда будем сохранять модели, и где будут лежать данные
    os.makedirs(base_dir)

In [4]:
texts = pd.read_csv(f'{base_dir}texts_and_metadata.txt', sep='\t')
texts.sample(3)

,document.id,source,stage,source_text,lemm_text,city,region,date
7290,735757619,iqbuzz,3,"❗❗Обращаюсь ко всем нацистам, которые имеют чт...",обращаться нацист иметь против армянский фамил...,Оренбург,Оренбургская область,2015-01-31
4863,1069095200,iqbuzz,2,"[[id49009300|Наталья], Ты думаешь они не сомне...",наталья думать сомневаться дать опрос полный д...,Санкт-Петербург,Санкт-Петербург,2015-10-24
14252,3713523,vk,1,Эдакий клип. Посвящен ветеранам различных войн...,эдакий клип посвящать ветеран различный война ...,NaN,default_region,2014-09-11


In [5]:
data = pd.read_csv(f'{base_dir}coding_results.txt', low_memory=False, sep='\t')
data.sample(3)

,document.id,source,stage,data,assessor,seed_eth_group,for_questions_about_text,do_text_make_sense_raw,do_text_make_sense_recoded,has_ethnonym_raw,...,represent_ethicity_raw,represent_ethicity_meaning,is_ethicity_superior_raw,is_ethicity_superior_meaning,is_ethicity_aggressor_raw,is_ethicity_aggressor_meaning,is_ethicity_dangerous_raw,is_ethicity_dangerous_meaning,comment,old_id
64746,1010604728,iqbuzz,2,2016-10-21 19:48:50,Tankly,карачаевец,0,yes,1,several,...,1.0,no,3.0,irrel,3.0,irrel,1.0,no,'Другой' = 'лазы'.,карачаевец_16
79058,702422676,iqbuzz,2,2016-09-17 20:42:25,skuchilina,уйгур,1,yes,1,several,...,1.0,no,3.0,irrel,3.0,irrel,1.0,no,NaN,уйгур_73
37439,878927408,iqbuzz,3,2017-04-01 00:45:38,an_men,литовец,0,yes,1,several,...,1.0,no,3.0,irrel,3.0,irrel,1.0,no,NaN,878927408


In [6]:
df = texts.merge(data, how='left')
df.sample(5)

,document.id,source,stage,source_text,lemm_text,city,region,date,data,assessor,...,represent_ethicity_raw,represent_ethicity_meaning,is_ethicity_superior_raw,is_ethicity_superior_meaning,is_ethicity_aggressor_raw,is_ethicity_aggressor_meaning,is_ethicity_dangerous_raw,is_ethicity_dangerous_meaning,comment,old_id
5354,869004589,iqbuzz,2,<b>Башкиры</b> -- один из древнейших народов м...,тюрк россия республика башкортостан башкир оди...,Москва,Москва,2015-04-21,2016-10-11 21:11:59,asukhanova,...,1.0,no,3.0,irrel,3.0,irrel,1.0,no,NaN,башкир_80
44300,733315158,iqbuzz,3,Сочи и Анапа – полный «пакет» от Coral Travel!...,сочи анапа полный пакет coral travel coral tra...,Стерлитамак,Красноярский край,2015-01-30,2017-03-24 12:54:34,an_men,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,733315158
15538,1000539562,iqbuzz,2,"[[id211685480|Заур], это связано с твоими слов...",заур связать твой слово некий исконность кавка...,Владикавказ,Северная Осетия - Алания,2015-08-21,2016-08-14 11:28:45,Tatiana,...,1.0,no,3.0,irrel,3.0,irrel,1.0,no,NaN,кумык_93
14981,454759880,iqbuzz,2,Исчезающие народы России Вскоре их можно будет...,исчезать народ россия вскоре можно увидеть уче...,NaN,NaN,2014-04-30,2016-08-30 00:03:58,Tatiana,...,1.0,no,3.0,irrel,3.0,irrel,1.0,no,NaN,коряк_76
15782,808427853,iqbuzz,2,Меня одного 15 <b>таджиков</b> побили на стади...,один таджик побить стадион ребята год клясться...,Москва,Москва,2015-03-14,2016-08-07 16:48:55,Tatiana,...,1.0,no,3.0,irrel,3.0,irrel,1.0,no,NaN,кыргыз_2


In [7]:
df.shape

(84784, 61)

In [8]:
df['do_text_make_sense_raw'].value_counts()

do_text_make_sense_raw
yes     78372
no       4334
lang     1890
joke      188
Name: count, dtype: int64

In [9]:
df.drop(df[df['do_text_make_sense_raw'] == 'no'].index, inplace=True)

In [10]:
ops = ['is_text_positive_recoded', 'is_text_neg_recoded']
df[ops] = df[ops].apply(pd.to_numeric, errors='coerce')
pos_ = df['is_text_positive_recoded']
neg_ = df['is_text_neg_recoded']
df.loc[(pos_ > 0) & (neg_ < 0), ops] = None
df.loc[neg_ < 0, 'text_sentiment'] = -1
df.loc[pos_ > 0, 'text_sentiment'] = 1
df.loc[(neg_ == 0) & (pos_ == 0), 'text_sentiment'] = 0

In [11]:
MAX_SPOIL = len(args)
MAX_SPOIL

7

In [12]:
df['has_pos_eth_interaction_raw'].value_counts()

has_pos_eth_interaction_raw
no     62783
yes    13603
unk     1986
Name: count, dtype: int64

In [14]:
df = df.fillna(np.nan).replace([np.nan], [None])

In [15]:
ids = df['document.id'].unique()
ids.shape  # 14196 после дропа по do_text_make_sence = no

(14196,)

In [16]:
def clean(text):
    text = text.apply(lambda x: str(x))
    CLEANR = [re.compile('<.*?>'), re.compile("\[.*?\]")]
    for i in CLEANR:
        text = text.apply(lambda x: re.sub(i, '', x))
    text = text.apply(lambda x: x.replace('\\', ''))
    text = text.apply(lambda x: re.sub(r"([" + re.escape(string.punctuation) + r"])\1+", r"\1", x))
    text = text.apply(lambda x: re.sub(r"http\S+", '', x))
    text = text.apply(lambda x: re.sub(r"\r", '', x))
    text = text.apply(lambda x: re.sub(r'\s+', ' ', x))
    text = text.apply(lambda x: re.sub(r'\s+', ' ', x))
    text = text.apply(lambda x: re.sub('"+','"', x))
    text = text.apply(lambda x: re.sub("'+","'", x))
    items = string.punctuation + " "
    text = text.apply(lambda x: x.lstrip(items) if isinstance(x, str) else x)
    return text

<>:3: SyntaxWarning: invalid escape sequence '\['
<>:3: SyntaxWarning: invalid escape sequence '\['
C:\Users\bentodan\AppData\Local\Temp\ipykernel_18976\3087058704.py:3: SyntaxWarning: invalid escape sequence '\['
  CLEANR = [re.compile('<.*?>'), re.compile("\[.*?\]")]


In [17]:
df['source_text'] = clean(df['source_text'])

In [18]:
df.to_csv("data/df.csv", index=False)

In [18]:
df['source_text'][0]

'но у вас же бред написан. Какими русскими? Вообще то там грузины воевали с Абхазами. Это исторический факт. А статья может быть на 10% правдива. Почему вы верите, какой-то статье?'

In [19]:
preprocessor = Preprocessor.Preprocessor(df=df, args=args, var_vocab=var_vocab, topic_to_russian=topic_to_russian)

In [20]:
id_ = df['document.id'].sample().values[0]
descr, text = preprocessor.fit(id_)
print(id_, descr, text, sep='\n')

783920705
этот текст является позитивным, в тексте этнический конфликт отсутствует, в тексте не зафиксировано позитивного взаимодействия этичностей, мнение об этничности 'армянин' положительное, этничность 'армянин' не является опасной

5 марта в 19:00 Областной дом народного творчества приглашает всех на долгожданное событие – областной праздник-конкурс национальных красавиц Дона «Краса Юга России»! Праздник национальных культур и красоты проводится второй раз на территории Ростовской области. В празднике-конкурсе примут участие лучшие представительницы народов, проживающих на Дону. Армянскую диаспору Дона представит Петросян Лиана.Ей наша поддержка очень нужна. Давайте все вместе соберемся и пойдем болеть за нашу представительницу. Билеты стоят 300 руб


In [21]:
descr_spoiled, text = preprocessor.fit(id_, spoil_size=len(preprocessor.args))
print(descr, descr_spoiled, sep='\n')

этот текст является позитивным, в тексте этнический конфликт отсутствует, в тексте не зафиксировано позитивного взаимодействия этичностей, мнение об этничности 'армянин' положительное, этничность 'армянин' не является опасной

этот текст является негативным, мнение об этничности 'армянин' отрицательное, этничность 'армянин' является доминирующей, этничность 'армянин' является жертвой



In [22]:
descr_topics, _ = preprocessor.fit(id_, topic=True)
descr_topics_spoiled, _ = preprocessor.fit(id_, topic=True, topic_spoil=1)
print(descr_topics, descr_topics_spoiled, sep='\n')

Текст имеет темы: культура, этничность, этот текст является позитивным, в тексте этнический конфликт отсутствует, в тексте не зафиксировано позитивного взаимодействия этичностей, мнение об этничности 'армянин' положительное, этничность 'армянин' не является опасной

Текст имеет темы: экономика, история, этот текст является позитивным, в тексте этнический конфликт отсутствует, в тексте не зафиксировано позитивного взаимодействия этичностей, мнение об этничности 'армянин' положительное, этничность 'армянин' не является опасной



### Custom dataset <a class="anchor" id="c3"></a>

In [23]:
process_ids, test_ids = train_test_split(ids, test_size=0.2, random_state=RANDOM_STATE)
train_ids, validate_ids = train_test_split(process_ids, train_size=0.75, random_state=RANDOM_STATE)

train = df.loc[df['document.id'].isin(train_ids)]
test = df.loc[df['document.id'].isin(test_ids)]
validate = df.loc[df['document.id'].isin(validate_ids)]
train.shape, test.shape, validate.shape  # percents are ≈ (60%, 20%, 20%)

((47999, 62), (16170, 62), (16281, 62))

### Custom model <a class="anchor" id="c4"></a>

In [24]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(f"Device used: {device}.")

Device used: cpu.


In [25]:
model = MyModel.MyModel(device) # turn on the developer mode here
print(f"Model loaded. Model tokenizer is {model.tokenizer}.")

Model loaded. Model tokenizer is BertTokenizerFast(name_or_path='cointegrated/rubert-tiny2', vocab_size=83828, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	3: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	4: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}.


In [26]:
model.reinitialize()
print(f"Model reloaded. Model tokenizer is {model.tokenizer}.")

Model reloaded. Model tokenizer is BertTokenizerFast(name_or_path='cointegrated/rubert-tiny2', vocab_size=83828, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	3: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	4: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}.


### Trainer <a class="anchor" id="c5"></a>

In [27]:
params = {
    'batch_size': [8, 16],
    'lr': [1e-5, 1e-6, 1e-4],
    'max_spoil': range(1, MAX_SPOIL + 1, 2),
    'spoil_proba': np.arange(0.2,  0.8 + 0.1, 0.1)
}

In [28]:
trainer = Trainer.Trainer(MyModel.MyModel, device, train, validate, test, preprocessor=preprocessor, params=params)

### BERT training <a class="anchor" id="c6"></a>

In [29]:
torch.cuda.empty_cache()  # just in case

In [2]:
# trainer.choose_model() # uncomment if you want to find best model

In [3]:
# trainer.save()

In [ ]:
# trainer.plot_loss(on_train=False)

In [ ]:
# trainer.plot_metrics(on_train=False)

### GPT training <a class="anchor" id="c7"></a>

In [39]:
from gpt_classes import GPTDataset, GPTModel, GPTTrainer

In [40]:
gptdataset = GPTDataset.GPTDataset(df, preprocessor=preprocessor, args=var_vocab)

In [41]:
print(gptdataset[0][0])

Задание: Сгенерируй описание следующего текста, оцени тональность текста, оцени наличие этнического конфликта, наличие позитивного взаимодействия этничностей, мнение о абхаз в тексте, является ли этничность абхаз доминирующей, является ли этничность абхаз агрессором, является ли этничность абхаз опасной.
Текст: но у вас же бред написан. Какими русскими? Вообще то там грузины воевали с Абхазами. Это исторический факт. А статья может быть на 10% правдива. Почему вы верите, какой-то статье?
Описание: этот текст является негативным, в тексте есть этнический конфликт, в тексте не зафиксировано позитивного взаимодействия этичностей



In [42]:
from torch.utils.data import DataLoader

In [43]:
max_items = 100 # for now on
gptTrainer = GPTTrainer.GPTTrainer(GPTModel.GPTModel, device, train[:max_items], validate[:max_items], test[:max_items], preprocessor=preprocessor, dataset=GPTDataset.GPTDataset, args=var_vocab, dataloader=DataLoader)
# gptTrainer = GPTTrainer.GPTTrainer(GPTModel.GPTModel, device, train, validate, test, preprocessor=preprocessor, dataset=GPTDataset.GPTDataset, args=var_vocab, dataloader=DataLoader)

In [45]:
print(gptTrainer.model.my_generate('Что такое питон?')) # слон - это змея:)

Что такое питон?
Питон - это млекопитающее, которое питается плодами растений и животных.

Какие у вас ассоциации со словом "питон"?
Птица
птица, которая летает, летающая, летящая
Это птица. Птица - птица, которую можно увидеть в небе, а можно и не увидеть. Это птица-попугай, который живет на земле. А еще это птица - птичка, которой можно летать. И еще - птеродактиль


In [ ]:
gptTrainer.train(1)

In [ ]:
print(gptTrainer.model.my_generate('Что такое питон?'))

In [ ]:
for id_, generated in gptTrainer.data[0][0]['generated']:
    print(id_)
    print(generated)
    print("\n-----\n")
    break

In [ ]:
print(gptTrainer.data[0][0]['matched_with_descr'])

In [ ]:
GPTparams = {
    'cut_data': [True, False],
    'batch_size': [10, 20],
    'lr': [1e-5, 1e-6, 1e-4],
    'ngrams': [1, 2, 3, 4, 5]
}

In [ ]:
gptTrainer.choose_model(GPTparams)